# Preprocessing Dataset

In [22]:
# Step 1: Preprocessing the Dataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy


In [23]:
# Load the dataset
file_path = r"D:\UOG\PROJECTS\Threat\Submission 4\Submission-4\combined_opcodes.csv"
data = pd.read_csv(file_path)

# Replace NaN values in the Opcodes column with an empty string
data['Opcodes'] = data['Opcodes'].fillna('')

# Assign labels dynamically (example: assume "APT" indicates malicious)
def assign_label(file_name):
    if "APT" in file_name:  # Example: APT indicates malicious
        return 1
    else:
        return 0

data['Label'] = data['File Name'].apply(assign_label)

# Tokenize Opcodes and convert to binary indicator vectors
vectorizer = MultiLabelBinarizer()
data['Opcodes'] = data['Opcodes'].apply(lambda x: x.split(','))
X = vectorizer.fit_transform(data['Opcodes'])
y = data['Label'].values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Number of features (unique opcodes)
num_features = X_train.shape[1]

# TRAIN THE BASELINE NEURAL NETWORK 

In [24]:
# Define the baseline model
model = Sequential([
    Dense(64, activation='relu', input_dim=num_features),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=BinaryCrossentropy(), 
              metrics=['accuracy'])

# Train the baseline model
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=10, batch_size=32, verbose=2)

# Evaluate the baseline model
baseline_loss, baseline_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Baseline Model - Test Loss: {baseline_loss}, Test Accuracy: {baseline_acc}")

C:\Users\vidhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
5/5 - 2s - 381ms/step - accuracy: 0.6897 - loss: 0.6259 - val_accuracy: 0.9583 - val_loss: 0.4423
Epoch 2/10
5/5 - 0s - 92ms/step - accuracy: 0.9586 - loss: 0.3635 - val_accuracy: 0.9583 - val_loss: 0.2862
Epoch 3/10
5/5 - 0s - 36ms/step - accuracy: 0.9586 - loss: 0.2479 - val_accuracy: 0.9583 - val_loss: 0.2430
Epoch 4/10
5/5 - 0s - 42ms/step - accuracy: 0.9586 - loss: 0.2070 - val_accuracy: 0.9583 - val_loss: 0.2477
Epoch 5/10
5/5 - 0s - 29ms/step - accuracy: 0.9586 - loss: 0.2129 - val_accuracy: 0.9583 - val_loss: 0.2575
Epoch 6/10
5/5 - 0s - 35ms/step - accuracy: 0.9586 - loss: 0.1811 - val_accuracy: 0.9583 - val_loss: 0.2628
Epoch 7/10
5/5 - 0s - 34ms/step - accuracy: 0.9586 - loss: 0.1827 - val_accuracy: 0.9583 - val_loss: 0.2630
Epoch 8/10
5/5 - 0s - 35ms/step - accuracy: 0.9586 - loss: 0.1641 - val_accuracy: 0.9583 - val_loss: 0.2586
Epoch 9/10
5/5 - 0s - 34ms/step - accuracy: 0.9586 - loss: 0.1738 - val_accuracy: 0.9583 - val_loss: 0.2527
Epoch 10/10
5/5 - 0s - 30ms

# GENERATE ADVERSARIAL EXAMPLES

In [25]:
# Generate Adversarial Examples
def generate_pgd_examples(model, X, y, epsilon=0.02, alpha=0.005, iterations=5):
    """
    Generate adversarial examples using FGSM.

    Args:
        model: Trained neural network model.
        X: Input feature matrix.
        y: Labels for the input data.
        epsilon: Magnitude of the perturbation.

    Returns:
        X_adv: Adversarial examples.
    """
    X_adv = np.copy(X)
    for i in range(iterations):
        X_adv_tensor = tf.convert_to_tensor(X_adv, dtype=tf.float32)
        y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)

        # Reshape y_tensor to match the shape of y_pred
        y_tensor = tf.reshape(y_tensor, (-1, 1))  # Convert to shape (batch_size, 1)
        
        with tf.GradientTape() as tape:
            tape.watch(X_adv_tensor)
            y_pred = model(X_adv_tensor)
            loss = tf.keras.losses.binary_crossentropy(y_tensor, y_pred)

        # Compute gradients of loss w.r.t. inputs
        gradients = tape.gradient(loss, X_adv_tensor)

        # Apply perturbation
        X_adv = X_adv + alpha * np.sign(gradients.numpy())

        # Clip to maintain constraints
        X_adv = np.clip(X_adv, X - epsilon, X + epsilon)  # Project back to epsilon-ball
        X_adv = np.clip(X_adv, 0, 1)  # Ensure binary constraints

    return X_adv

# Generate adversarial examples for the training set
X_train_adv = generate_pgd_examples(model, X_train, y_train, epsilon=0.02, alpha=0.005, iterations=5)
print("Adversarial examples for training set generated.")

Adversarial examples for training set generated.


# ADVERSARIAL TRAINING 

In [26]:
# Adversarial Training
# Combine natural and adversarial samples
X_train_combined = np.vstack([X_train, X_train_adv])
y_train_combined = np.hstack([y_train, y_train])  # Duplicate labels for adversarial examples

# Define and compile the adversarially trained model
adv_model = Sequential([
    Dense(64, activation='relu', input_dim=num_features),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

adv_model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss=BinaryCrossentropy(), 
                  metrics=['accuracy'])

# Train the adversarially trained model
adv_history = adv_model.fit(X_train_combined, y_train_combined, 
                            validation_data=(X_val, y_val), 
                            epochs=10, batch_size=32, verbose=2)


Epoch 1/10
10/10 - 2s - 193ms/step - accuracy: 0.9552 - loss: 0.4028 - val_accuracy: 0.9583 - val_loss: 0.2792
Epoch 2/10
10/10 - 0s - 21ms/step - accuracy: 0.9586 - loss: 0.2196 - val_accuracy: 0.9583 - val_loss: 0.2618
Epoch 3/10
10/10 - 0s - 27ms/step - accuracy: 0.9586 - loss: 0.2114 - val_accuracy: 0.9583 - val_loss: 0.2769
Epoch 4/10
10/10 - 0s - 22ms/step - accuracy: 0.9586 - loss: 0.1844 - val_accuracy: 0.9583 - val_loss: 0.2761
Epoch 5/10
10/10 - 0s - 23ms/step - accuracy: 0.9586 - loss: 0.1831 - val_accuracy: 0.9583 - val_loss: 0.2629
Epoch 6/10
10/10 - 0s - 23ms/step - accuracy: 0.9586 - loss: 0.1534 - val_accuracy: 0.9583 - val_loss: 0.2648
Epoch 7/10
10/10 - 0s - 34ms/step - accuracy: 0.9586 - loss: 0.1290 - val_accuracy: 0.9583 - val_loss: 0.2748
Epoch 8/10
10/10 - 0s - 23ms/step - accuracy: 0.9586 - loss: 0.1192 - val_accuracy: 0.9583 - val_loss: 0.2968
Epoch 9/10
10/10 - 0s - 19ms/step - accuracy: 0.9586 - loss: 0.1044 - val_accuracy: 0.9583 - val_loss: 0.3299
Epoch 10/

# EVALUATE MODEL ROBUSTNESS 

In [27]:
# Evaluate on natural test samples
natural_loss, natural_acc = adv_model.evaluate(X_test, y_test, verbose=0)

# Generate adversarial test samples
X_test_adv = generate_pgd_examples(adv_model, X_test, y_test, epsilon=0.02, alpha=0.005, iterations=5)

# Evaluate on adversarial test samples
adv_loss, adv_acc = adv_model.evaluate(X_test_adv, y_test, verbose=0)

print(f"Natural Test Accuracy: {natural_acc * 100:.2f}%")
print(f"Adversarial Test Accuracy: {adv_acc * 100:.2f}%")
print(f"Clean Loss: {natural_loss}, Adversarial Loss: {adv_loss}")

Natural Test Accuracy: 95.92%
Adversarial Test Accuracy: 95.92%
Clean Loss: 0.2649163603782654, Adversarial Loss: 0.383309543132782
